# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298338691278                   -0.85    5.6
  2   -8.300190365277       -2.73       -1.25    1.0
  3   -8.300429151955       -3.62       -1.88    2.9
  4   -8.300461855841       -4.49       -2.74    2.0
  5   -8.300463883625       -5.69       -3.02    3.1
  6   -8.300464256302       -6.43       -3.19    8.0
  7   -8.300464460099       -6.69       -3.36    1.2
  8   -8.300464552885       -7.03       -3.50    6.0
  9   -8.300464621550       -7.16       -3.71    2.4
 10   -8.300464625622       -8.39       -3.77    1.0
 11   -8.300464640632       -7.82       -4.03    1.8


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67458546507                   -0.70    5.9
  2   -16.67869776527       -2.39       -1.14    1.4
  3   -16.67920368334       -3.30       -1.87    2.2
  4   -16.67927996294       -4.12       -2.75    3.2
  5   -16.67928590175       -5.23       -3.17    6.5
  6   -16.67928618239       -6.55       -3.46    2.2
  7   -16.67928621635       -7.47       -3.90    2.4
  8   -16.67928622392       -8.12       -4.54    2.6


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32566005547                   -0.56    6.4
  2   -33.33278820076       -2.15       -1.00    1.0
  3   -33.33412242679       -2.87       -1.73    5.5
  4   -33.33427892990       -3.81       -2.64    2.0
  5   -33.33608863241       -2.74       -2.41    5.8
  6   -33.33694304138       -3.07       -2.52    7.9
  7   -33.33694307334       -7.50       -2.53    1.0
  8   -33.33682091410   +   -3.91       -2.28    2.0
  9   -33.33678084796   +   -4.40       -2.22    1.0
 10   -33.33678314082       -5.64       -2.23    1.0
 11   -33.33681200970       -4.54       -2.27    1.0
 12   -33.33681848961       -5.19       -2.28    1.0
 13   -33.33683062286       -4.92       -2.31    1.0
 14   -33.33686669748       -4.44       -2.39    1.0
 15   -33.33684298064   +   -4.62       -2.33    1.0
 16   -33.33688285240       -4.40       -2.44    1.0
 17   -33.33694344267       -4.22       -3.57 

When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298310582971                   -0.85    4.9
  2   -8.300266375133       -2.71       -1.59    1.8
  3   -8.300428752736       -3.79       -2.58    2.8
  4   -8.300392283795   +   -4.44       -2.32    3.1
  5   -8.300464207645       -4.14       -3.37    1.1
  6   -8.300464575547       -6.43       -3.77    2.4
  7   -8.300464640129       -7.19       -4.25    2.2


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32583095842                   -0.56    7.5
  2   -33.32274661083   +   -2.51       -1.27    1.5
  3   -19.57486528287   +    1.14       -0.50    6.8
  4   -33.30924117750        1.14       -1.64    5.2
  5   -33.21511879243   +   -1.03       -1.31    2.8
  6   -31.62240623792   +    0.20       -0.95    4.1
  7   -33.32957835844        0.23       -2.02    4.6
  8   -33.33474410785       -2.29       -2.23    2.1
  9   -33.33666750398       -2.72       -2.63    2.0
 10   -33.33663370371   +   -4.47       -2.78    2.0
 11   -33.33692792751       -3.53       -3.09    3.1
 12   -33.33693768127       -5.01       -3.55    2.8
 13   -33.33694297553       -5.28       -3.88    3.1
 14   -33.33694375463       -6.11       -4.05    3.4


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.